Decrypt passwords.json?

In [ ]:
var path = require('path');
var rfs = require('fs').readFileSync;
var crypto = require('crypto');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PASS_FILE = path.join(PROFILE_PATH, '.credentials', 'password.txt');
var PASSWORDS_FILE = path.join(PROFILE_PATH, '.credentials', 'passwords.json');
var pass = process.env.SELENIUM_PASS || rfs(PASS_FILE);

var decrypt = (text) => {
    var decipher = crypto.createDecipher('aes-256-ctr', pass);
    var dec = decipher.update(text, 'hex', 'latin1');
    dec += decipher.final('latin1');
    return dec;
}

var getCredentials = (name) => {
    var resultSet = {};
    var passwords = JSON.parse(rfs(PASSWORDS_FILE));
    var set = passwords.filter(el => el.host == name)[0] || {};
    for(var i in set) {
        if(set.hasOwnProperty(i)) {
            if(i == 'added' || i == 'host') {
                resultSet[i] = set[i];
                continue;
            }
            resultSet[i] = decrypt(set[i]);
        }
    }
    return resultSet;
};
module.exports = getCredentials;
getCredentials;



Add encrypted to passwords.json?



In [ ]:
var fs = require('fs');
var rfs = require('fs').readFileSync;
var crypto = require('crypto');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PASS_FILE = path.join(PROFILE_PATH, '.credentials', 'password.txt');
var PASSWORDS_FILE = path.join(PROFILE_PATH, '.credentials', 'passwords.json');
var pass = process.env.SELENIUM_PASS || rfs(PASS_FILE);

function encrypt(text) {
    var cipher = crypto.createCipher('aes-256-ctr', pass);
    var crypted = cipher.update(text, 'latin1', 'hex');
    crypted += cipher.final('hex');
    return crypted;
}

function encryptSet(set) {
    var resultSet = {};
    for(var i in set) {
        if(set.hasOwnProperty(i)) {
            if(i == 'added' || i == 'host') {
                resultSet[i] = set[i];
                continue;
            }
            resultSet[i] = encrypt(set[i]);
        }
    }
    return resultSet;
}

function saveCredentials(passwordAddJson) {
    // TODO: passAdd from file?
    var passwords = JSON.parse(fs.readFileSync(PASSWORDS_FILE));
    passwordAddJson.added = new Date();
    var encrypted = encryptSet(passwordAddJson);
    if(passwordAddJson.host) {
        passwords = passwords.filter(function (el) {
            return el.host.toLowerCase() != passwordAddJson.host.toLowerCase();
        });
    }
    passwords[passwords.length] = encrypted;
    var passwordString = JSON.stringify(passwords, null, 4);
    //var compressed = zlib.gzipSync(passwordString).toString('base64');
    console.log('Passwords: Saving encrypted file');
    fs.writeFileSync('passwords-new.json', passwordString);
    fs.renameSync(PASSWORDS_FILE, 'passwords-backup.json');
    fs.renameSync('passwords-new.json', PASSWORDS_FILE);
    fs.unlinkSync('passwords-backup.json');
    if(typeof passAdd !== 'undefined' && fs.existsSync(passAdd)) {
        console.log('Passwords: Deleting input file: ' + passAdd);
        fs.unlinkSync(passAdd);
    }
    return encrypted;
}
module.exports = saveCredentials;
saveCredentials;


In [ ]:
// add all passwords from passwords.html?
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var fs = require('fs');
var project = PROFILE_PATH + '/Documents/passwords.htm';

// How to convert a string to an Array of RegEx matches
var regexToArray = (ex, str, i = 0) => {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i]));
    return co;
};

var readPasswordsHtm = (project) => {
    return importer.interpretAll(['add encrypted passwords.json'])
    .then(r => r[0].runInNewContext())
    .then(saveCredentials => {
        var credentials;
        var passwords = fs.readFileSync(project).toString('utf16le');
        regexToArray(/TBODY[^>]*>[\s\S]*?\/TBODY/ig, passwords).forEach(b => {
            var creds = {};
            creds['host'] = (((/subcaption[^>]*>(.*?)<\/td>/ig)
                .exec(b) || [])[1] || '').replace(/<wbr>/ig, '').toLowerCase();
            if(creds.host.trim() == '') {
                return;
            }
            regexToArray(/<tr>[\s\S]*?<\/tr>/ig, b).forEach(f => {
                var key = (((/field[^>]*>(.*?)<\/td>/ig)
                    .exec(f) || [])[1] || '').replace(/<wbr>/ig, '');
                var value = (((/wordbreakfield[^>]*>(.*?)<\/td>/ig)
                    .exec(f) || [])[1] || '').replace(/<wbr>/ig, '');
                if(key.trim() !== '') {
                    creds[key] = value;
                }
            });
            credentials = saveCredentials(creds);
        });
        return credentials;
    })
};
module.exports = readPasswordsHtm;
readPasswordsHtm;
   


// display login form and add to passwords.json?



download passwords from https://passwords.google.com ?



In [2]:
var importer = require('../Core');

var downloadGooglePasswords = () => {
    return client.url('https://passwords.google.com')
    .then(() => importer.interpretAll(['log in google']))
    .then(r => r[0].runInNewContext({client, getCredentials}))
    .then(() => client.loginGoogle())
    .pause(1000)
    .elements('[role="row"]')
    .then(els => els.value.length)
    .then(length => {
        for(var i = 0; i < length; i++) {
            client.click(
                '[role="row"]:nth-of-type('+i+') [role="rowheader"] ~ [role="gridcell"]:last-child [role="button"]')
            .pause(1000)
            .then(() => Promise.all([
                client.getText('[role="row"]:nth-of-type('+i+') [role="rowheader"]'),
                client.getText('[role="row"]:nth-of-type('+i+') [role="rowheader"] ~ [role="gridcell"]'),
                client.getValue('[role="row"]:nth-of-type('+i+') [role="rowheader"] ~ [role="gridcell"]:last-child input'),
            ]))
            .then(r => saveCredentials({
                host: r[0],
                username: r[1],
                password: r[2],
            }))
        }
        return client;
    })
    ;

};
module.exports = downloadGooglePasswords;
downloadGooglePasswords;


[Function: downloadGooglePasswords]

In [3]:
var importer = require('../Core');

$$.async();
var runSeleniumCell;
importer.interpretAll(['add encrypted passwords.json', 'selenium cell'])
.then(r => {
    var saveCredentials = r[0].runInNewContext();
    runSeleniumCell = r[1].runInNewContext({saveCredentials});
    return runSeleniumCell('download passwords from google');
})
.then(downloadGooglePasswords => downloadGooglePasswords())
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Google: Require password
length 130


How to log in to Google using webdriver?

In [ ]:
var client = typeof client !== undefined ? client : {};
var matchUsername = 'input[name="Email"], input[autocomplete="username"]';
var enterGoogleUsername = (email) => {
    console.log('Google: Sign in required');
    return client
        .waitForVisible(matchUsername, 1000)
        .addValue(matchUsername, email)
        .click('#identifierNext');
}
if (typeof client.enterGoogleUsername == 'undefined') {
    client.addCommand('enterGoogleUsername', enterGoogleUsername);
}

var matchPassword = 'input[name="Passwd"], input[autocomplete="password"], input[name="password"]';
var enterGooglePassword = (pass) => {
    console.log('Google: Require password');
    return client
        .waitForVisible(matchPassword, 5000)
        .addValue(matchPassword, pass)
        .click('#passwordNext');
}
if (typeof client.enterGooglePassword == 'undefined') {
    client.addCommand('enterGooglePassword', enterGooglePassword);
}

var enterCredentials = (is) => {
    var credentials = getCredentials('accounts.google.com');
    if (is) {
        return client
            .isVisible('#passwordNext').then(is => is 
                ? client.enterGooglePassword(credentials.Passwd)
                : client.enterGoogleUsername(credentials.Email)
                        .enterGooglePassword(credentials.Passwd))
    } else {
        return client.isExisting('h1*=Choose an account')
            .then(is => {
                if (is) {
                    return client
                        .click('p*=' + credentials.Email);
                }
            });
    }
}
if (typeof client.enterCredentials == 'undefined') {
    client.addCommand('enterCredentials', enterCredentials);
}

var loginGoogle = () => {
    return client.isExisting('form[action*="signin"]')
        .then(is => client.enterCredentials(is))
        .catch((e) => {
            console.log(e);
            console.log('Google: Could not log in');
        });
}
if (typeof client.loginGoogle == 'undefined') {
    client.addCommand('loginGoogle', loginGoogle);
}
module.exports = loginGoogle;
loginGoogle;

